# Dense Layer implementation from scratch #

In [6]:
import numpy as np
import copy
import math

np.random.seed(42)

class Layer(object):
    def set_input_shape(self, shape):
        self.input_shape = shape
    
    def layer_name(self):
        return self.__class__.__name__
    
    def parameters(self):
        return 0
    
    def forward_pass(self, X, training):
        raise NotImplementedError()
    
    def backward_pass(self, accum_grad):
        raise NotImplementedError()
    
    def output_shape(self):
        raise NotImplementedError()

class Dense(Layer):
    def __init__(self, n_units, input_shape=None):
        self.layer_input = None
        self.input_shape = input_shape
        self.n_units = n_units
        self.trainable = True
        self.W = None
        self.w0 = None   
    
    def initialize(self, optimizer):
        limit = 1 / math.sqrt(self.input_shape[0])
        self.W = np.random.uniform(-limit,limit,(self.input_shape[0],self.n_units)) 
        self.w0 = np.zeros(self.n_units)   
        self.W_opt = copy.deepcopy(optimizer)
        self.w0_opt = copy.deepcopy(optimizer)
    
    def forward_pass(self, X):
        self.layer_input = X
        return X @ self.W + self.w0
    
    def backward_pass(self, accum_grad):
        dX = accum_grad @ self.W.T
        dW = self.layer_input.T @ accum_grad
        dw0 = np.sum(accum_grad, axis = 0)
        if self.trainable:
            self.W = self.W_opt.update(self.W, dW)
            self.w0 = self.w0_opt.update(self.w0, dw0)
        return dX
    
    def output_shape(self):
        return (self.n_units,)
    def number_of_parameters():
        return np.prod(self.W.shape) + np.prod(self.w0.shape)


#Simple SGD optimiser for testing
class SimpleOptimizer:
    def update(self, weights, grad):
        return weights - 0.01 * grad  

dense = Dense(n_units=3, input_shape=(2,))
dense.initialize(SimpleOptimizer())

X = np.array([[1.0, 2.0]]) 
output = dense.forward_pass(X)
print("Forward pass output:", output)

grad = np.array([[0.1, 0.2, 0.3]])  
dX = dense.backward_pass(grad)
print("Backward pass output (dX):", dX)
print("Updated W:", dense.W)
print("Updated w0:", dense.w0)

Forward pass output: [[ 0.10162127 -0.33551992 -0.64490545]]
Backward pass output (dX): [[ 0.20816524 -0.22928937]]
Updated W: [[-0.17842707  0.63540628  0.32508898]
 [ 0.13752417 -0.4904631  -0.49249721]]
Updated w0: [-0.001 -0.002 -0.003]
